In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [2]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [3]:
def kron(A, B):
    return la.kron(A, B)

In [4]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [5]:
print(kronsum(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 2.00000000  0.00000000  4.00000000  0.00000000]
 [ 0.00000000  2.00000000  0.00000000  4.00000000]
 [ 6.00000000  0.00000000  8.00000000  0.00000000]
 [ 0.00000000  6.00000000  0.00000000  8.00000000]]


In [6]:
n = 3
p_num = 100
eps_G = 10 ** (-8)
eps_Phi = 10 ** (-6)
eps_p_i = 10 ** (-6)

In [7]:
# Входной BMAP
matrD_0 = np.array([[-86., 0.01], [0.02, -2.76]]) / 7
matrD = np.array([[85, 0.99], [0.2, 2.54]]) / 7
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

for matr in matrD_k:
    print(matr)

[[-12.28571429  0.00142857]
 [ 0.00285714 -0.39428571]]
[[ 4.97658080  0.05796253]
 [ 0.01170960  0.14871194]]
[[ 3.98126464  0.04637002]
 [ 0.00936768  0.11896956]]
[[ 3.18501171  0.03709602]
 [ 0.00749415  0.09517564]]


In [8]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
print(matrD_1_)
matr_a = copy.deepcopy(matrD_1_)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)

matrdD_1_ = copy.deepcopy(matrD_k[1])

print(theta)

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = np.dot(np.dot(theta, matrdD_1_), vect_e)
print(lamD)

[[-0.14285714  0.14285714]
 [ 0.03142857 -0.03142857]]
[ 0.18032787  0.81967213]
[ 4.69791416]


In [9]:
# Поток поломок MAP
matrH0 = np.array([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.array([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = np.dot(np.dot(gamma, matrH1), vect_e)
print(h)

[[-0.05392500  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  1.00000000]
 [ 0.05392500 -0.14662500]]
[ 6.00065225]


In [10]:
# Поток обслуживания PH1
beta1 = np.matrix([1, 0])
matrS1 = np.matrix([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1], [1]])
matrS1_0 = - matrS1 * M1_e
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
print(np.dot(beta1, la.inv(matrS1)))
mu_1 = -la.inv(np.dot(np.dot(beta1, la.inv(matrS1)), vect_e))
print(mu_1)

[[-0.05000000 -0.05000000]]
[[ 10.00000000]]


In [11]:
# Поток обслуживания PH2
beta2 = np.matrix([1, 0])
matrS2 = np.matrix([[-2, 2], [0, -2]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1], [1]])
matrS2_0 = np.dot(- matrS2, M2_e)

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(np.dot(np.dot(beta2, la.inv(matrS2)), vect_e))
print(mu_2)

[[ 1.00000000]]


In [11]:
matrS_w = kron(np.dot(matrS1_0, beta1), 
               np.dot(M2_e, beta2)) + kron(np.dot(M1_e, beta1), np.dot(matrS2_0, beta2))

In [13]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([0.05, 0.95])
matrA1 = np.matrix([[-1.86075, 0.], [0., -146.9994]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1], [1]])
matrA1_0 = - np.dot(matrA1, L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(np.dot(np.dot(alpha1, la.inv(matrA1)), vect_e))
print(kappa_1)

[[ 29.99985287]]


In [14]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([0.05, 0.95])
matrA2 = np.matrix([[-5.58225, 0.], [0., -440.9982]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1], [1]])
matrA2_0 = - np.dot(matrA2, L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(np.dot(np.dot(alpha2, la.inv(matrA2)), vect_e))
print(kappa_2)

[[ 89.99955862]]


In [15]:
# Поток ремонта PH
tau = np.matrix([0.98, 0.02])
matrT = np.matrix([[-100., 0.], [0., -0.0002]])
T_e = np.matrix([[1], [1]])
matrT0 = - np.dot(matrT, T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -np.dot(np.dot(tau, la.inv(matrT)), vect_e)
print(phi)

[[ 100.00980000]]


In [16]:
a = W_ * V_

In [17]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0)

36
[[-20.39643929 -0.00000000  0.00142857  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.39478320  7.50088080  0.00805680  0.15307920
   0.00264233  0.05020417  0.00005393  0.00102458  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [-0.00000000 -20.39643929  0.00000000  0.00142857  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00718463  0.13650788  0.00014663  0.00278588
   0.00571463  0.10857788  0.00011663  0.00221588  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00285714  0.00000000 -0.65753571 -0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00

In [18]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)

for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    print('matrQw_k[' + str(i) + '] = ', sum)

matrQw_k[0] =  8.881784197e-16
matrQw_k[1] =  -7.847475
matrQw_k[2] =  7.847475
matrQw_k[3] =  1.11022302463e-16
matrQw_k[4] =  -4.4408920985e-15
matrQw_k[5] =  435.41595
matrQw_k[6] =  -7.847475
matrQw_k[7] =  427.568475
matrQw_k[8] =  -427.568475
matrQw_k[9] =  7.847475
matrQw_k[10] =  -435.41595
matrQw_k[11] =  1.42108547152e-14
matrQw_k[12] =  2.6645352591e-15
matrQw_k[13] =  -99.9998
matrQw_k[14] =  2.6645352591e-15
matrQw_k[15] =  -99.9998
matrQw_k[16] =  99.9998
matrQw_k[17] =  1.66533453694e-16
matrQw_k[18] =  99.9998
matrQw_k[19] =  1.66533453694e-16
matrQw_k[20] =  -8.881784197e-16
matrQw_k[21] =  145.13865
matrQw_k[22] =  -99.9998
matrQw_k[23] =  45.13885
matrQw_k[24] =  6.2172489379e-15
matrQw_k[25] =  145.13865
matrQw_k[26] =  -99.9998
matrQw_k[27] =  45.13885
matrQw_k[28] =  -45.13885
matrQw_k[29] =  99.9998
matrQw_k[30] =  -145.13865
matrQw_k[31] =  2.31481500634e-14
matrQw_k[32] =  -45.13885
matrQw_k[33] =  99.9998
matrQw_k[34] =  -145.13865
matrQw_k[35] =  2.3148150063

In [19]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)

print(matrQv_0)

[[ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 20.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.000000

In [20]:
# Q_0
block0 = kron(np.eye(a), np.dot(matrS1_0, beta1))
block1 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)

print(matrQ_0)

[[ 0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000]
 [ 20.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000

In [21]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])

print(matrQ_1)

[[-40.39643929 -0.00000000 -0.00000000 -0.00000000  20.00142857  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.39478320  7.50088080
   0.00805680  0.15307920  0.00264233  0.05020417  0.00005393  0.00102458
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000]
 [-0.00000000 -40.39643929 -0.00000000 -0.00000000  0.00000000  20.00142857
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00

In [22]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+2):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for j in range(1, np.array(matrQ_k).shape[0]):
        sum += np.sum(matrQ_k[j][i])
    print('matrQ_k[' + str(i) + '] = ', sum)

matrQ_k[0] =  2.6645352591e-15
matrQ_k[1] =  20.0
matrQ_k[2] =  -7.847475
matrQ_k[3] =  12.152525
matrQ_k[4] =  -12.152525
matrQ_k[5] =  7.847475
matrQ_k[6] =  -20.0
matrQ_k[7] =  1.83186799063e-15
matrQ_k[8] =  8.881784197e-16
matrQ_k[9] =  435.41595
matrQ_k[10] =  2.0
matrQ_k[11] =  437.41595
matrQ_k[12] =  -7.847475
matrQ_k[13] =  427.568475
matrQ_k[14] =  -5.847475
matrQ_k[15] =  429.568475
matrQ_k[16] =  -429.568475
matrQ_k[17] =  5.847475
matrQ_k[18] =  -427.568475
matrQ_k[19] =  7.847475
matrQ_k[20] =  -437.41595
matrQ_k[21] =  -2.0
matrQ_k[22] =  -435.41595
matrQ_k[23] =  -5.27355936697e-15
matrQ_k[24] =  -8.881784197e-16
matrQ_k[25] =  -99.9998
matrQ_k[26] =  2.0
matrQ_k[27] =  -97.9998
matrQ_k[28] =  -8.881784197e-16
matrQ_k[29] =  -99.9998
matrQ_k[30] =  2.0
matrQ_k[31] =  -97.9998
matrQ_k[32] =  97.9998
matrQ_k[33] =  -2.0
matrQ_k[34] =  99.9998
matrQ_k[35] =  5.55111512313e-17
matrQ_k[36] =  97.9998
matrQ_k[37] =  -2.0
matrQ_k[38] =  99.9998
matrQ_k[39] =  5.55111512313e-1

In [23]:
# Проверка генератора Q

In [40]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])


def calc_G(matrG_prev):
    temp_sum = copy.deepcopy(matrQ_k[0]) 
    for k in range(2, n + 2):
        temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG_prev, k))
    matrG_new = np.dot(matrQ_1_neg_inv, temp_sum)
    return matrG_new

matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)


i = 1
s = np.eye(56)
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    s = np.zeros((56, 56))
    for i in range(0, n + 2):
        s += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG, k))
    i += 1
    print(i, 'norm s = ', la.norm(s, ord=np.inf))

pd.DataFrame(matrG)
print(la.norm(matrG, ord=np.inf))

5 norm s =  0.09363642814708104
5 norm s =  4.042166946616536e-06
5 norm s =  4.8993076376623997e-14
5 norm s =  9.304696348536265e-30
5 norm s =  1.9594107652634713e-50
5 norm s =  1.959753415884916e-50
3.310180388318945


In [34]:
la.norm(matrG, ord=np.inf)

3.310180388318945

In [26]:
for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    print('Sum matrQw_k[' + str(i) + '] = ', sum)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for matr in matrQ_k:
        sum += np.sum(matr[i])
    print('Sum matrQ_k[' + str(i) + '] = ', sum)

Sum matrQw_k[0] =  8.881784197e-16
Sum matrQw_k[1] =  -7.847475
Sum matrQw_k[2] =  7.847475
Sum matrQw_k[3] =  1.11022302463e-16
Sum matrQw_k[4] =  -4.4408920985e-15
Sum matrQw_k[5] =  435.41595
Sum matrQw_k[6] =  -7.847475
Sum matrQw_k[7] =  427.568475
Sum matrQw_k[8] =  -427.568475
Sum matrQw_k[9] =  7.847475
Sum matrQw_k[10] =  -435.41595
Sum matrQw_k[11] =  1.42108547152e-14
Sum matrQw_k[12] =  2.6645352591e-15
Sum matrQw_k[13] =  -99.9998
Sum matrQw_k[14] =  2.6645352591e-15
Sum matrQw_k[15] =  -99.9998
Sum matrQw_k[16] =  99.9998
Sum matrQw_k[17] =  1.66533453694e-16
Sum matrQw_k[18] =  99.9998
Sum matrQw_k[19] =  1.66533453694e-16
Sum matrQw_k[20] =  -8.881784197e-16
Sum matrQw_k[21] =  145.13865
Sum matrQw_k[22] =  -99.9998
Sum matrQw_k[23] =  45.13885
Sum matrQw_k[24] =  6.2172489379e-15
Sum matrQw_k[25] =  145.13865
Sum matrQw_k[26] =  -99.9998
Sum matrQw_k[27] =  45.13885
Sum matrQw_k[28] =  -45.13885
Sum matrQw_k[29] =  99.9998
Sum matrQw_k[30] =  -145.13865
Sum matrQw_k[31

In [27]:
temp_sum = matrQ_k[1].copy()
for k in range(2, n+2):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = (-matrG_0).dot(matrQv_0)
pd.DataFrame(matrG_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.000000,0.000000,0.000000,0.000000,1.991200e-23,3.966372e-22,6.909025e-25,1.250264e-23,7.278490e-22,1.931235e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.495127,0.000000,0.479401,0.000000,-1.077167e-21,-2.044597e-20,-5.624130e-24,-1.063252e-22,-3.894346e-20,-5.421011e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,-2.683187e-23,-5.110041e-22,-6.389514e-24,-3.065243e-23,-9.722436e-22,-3.021037e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.495127,0.000000,0.479401,-3.440970e-23,-6.550232e-22,-3.452402e-24,-5.434897e-23,-1.245807e-21,-3.580131e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,-2.916455e-21,-5.547450e-20,-2.117780e-23,-4.027761e-22,-1.050709e-19,-2.602085e-18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000068,0.000000,0.968236,0.000000,-2.264491e-21,-4.307835e-20,-1.530988e-23,-2.905572e-22,-8.180286e-20,-2.073537e-18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.000000,0.000000,-1.248673e-22,-2.372718e-21,-1.824515e-23,-4.335126e-22,-4.506609e-21,-8.796923e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000068,0.000000,0.968236,-5.485367e-23,-1.044459e-21,-2.158911e-24,-7.330396e-23,-1.982826e-21,-5.960389e-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,5.400401e-23,9.922539e-22,2.910578e-24,5.491799e-23,1.752588e-21,-2.979071e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,3.764027e-22,7.099678e-21,1.302949e-23,2.446093e-22,1.295254e-20,-2.628578e-19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = matrQw_k[l].copy()
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k].copy()
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n+1, p_num):
            matrQ_il[i].append(np.zeros(matrQ_il[i][l % n+1].shape)) 
    else:
        for l in range(0, p_num):
            if l >= i and l - i <= n+1:
                temp_matr = matrQ_k[l-i].copy()
                for k in range(l+1, n+2):
                    if (k - i <= n+1):
                        mult_matr = matrQ_k[k-i].copy()
                        for kk in range(k-1, l-1, -1):
                            if kk == 0:
                                mult_matr = mult_matr.dot(matrG_0)
                            else:
                                mult_matr = mult_matr.dot(matrG)
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))
    
print(len(matrQ_il[3]))
pd.DataFrame(la.inv(-matrQ_il[1][1]))

100


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,2.217776e+48,0.000000e+00,-1.862626e+51,3.769764e+27,1.047042e+50,0.000000e+00,-5.249376e+50,-1.506696e+30,1.021826e+60,-2.507717e+59,...,-7.895751e+58,4.559359e+58,4.589739e+48,-3.405679e+49,-5.096000e+54,1.216456e+54,-4.331944e+49,4.954541e+50,1.327606e+56,-3.174374e+56
1,1.293537e+120,-1.233732e+101,-3.207256e+122,1.304287e+99,2.535531e+121,4.929370e+103,-1.087468e+122,-5.207366e+101,1.613004e+131,-1.067545e+131,...,-1.940966e+130,1.387252e+130,-2.785873e+120,-8.315708e+120,-1.262191e+126,2.052222e+125,-2.406920e+121,8.755816e+121,5.534797e+127,-8.212840e+127
2,-1.447808e+47,0.000000e+00,-2.497915e+49,1.249721e+25,1.237821e+48,0.000000e+00,-1.081360e+49,-5.001995e+27,-7.129664e+56,-4.062938e+57,...,-7.342802e+56,3.100251e+56,2.291152e+47,-4.639016e+47,-4.724837e+52,1.129037e+52,-1.373229e+48,5.964590e+48,2.025728e+53,-2.040665e+54
3,1.742906e+110,0.000000e+00,-6.313402e+111,3.663096e+88,7.738526e+110,0.000000e+00,-8.952193e+110,-1.463513e+91,-3.437521e+120,5.148699e+119,...,-7.723357e+119,6.572042e+119,-2.009158e+110,-1.777479e+110,-4.274685e+115,1.207900e+115,-1.373797e+110,1.739819e+111,2.644449e+117,-3.346658e+117
4,2.716384e+49,0.000000e+00,1.807870e+52,-1.813389e+27,-6.249819e+50,0.000000e+00,4.114613e+51,7.310036e+29,-1.070019e+61,-9.613554e+59,...,4.558531e+59,-1.209243e+59,-2.355440e+50,1.996289e+50,2.891540e+55,-1.203338e+55,-4.666448e+50,-4.688944e+51,4.465279e+56,1.614007e+57
5,-3.009234e+121,2.870106e+102,7.461235e+123,-3.039082e+100,-5.898560e+122,-1.146749e+105,2.529842e+123,1.213159e+103,-3.752429e+132,2.483495e+132,...,4.515388e+131,-3.227248e+131,6.480946e+121,1.934534e+122,2.936311e+127,-4.774210e+126,5.599365e+122,-2.036919e+123,-1.287593e+129,1.910603e+129
6,2.658019e+48,0.000000e+00,4.585900e+50,5.990227e+26,-2.272505e+49,0.000000e+00,1.985259e+50,-2.393867e+29,1.308929e+58,7.459113e+58,...,1.348059e+58,-5.691723e+57,-4.206307e+48,8.516730e+48,8.674287e+53,-2.072789e+53,2.521098e+49,-1.095034e+50,-3.719016e+54,3.746440e+55
7,-4.067596e+111,0.000000e+00,1.482725e+113,-8.547743e+89,-1.808708e+112,0.000000e+00,2.081785e+112,3.414900e+92,7.894757e+121,-1.194235e+121,...,1.804013e+121,-1.533414e+121,4.680150e+111,4.160489e+111,1.001747e+117,-2.827929e+116,3.131707e+111,-4.091672e+112,-6.173893e+118,7.828438e+118
8,-5.573743e+21,0.000000e+00,2.864266e+23,3.222198e-01,-1.278702e+22,0.000000e+00,7.650767e+22,-1.297148e+02,2.830247e+31,4.334579e+31,...,-3.769381e+30,9.078066e+30,-4.800202e+21,1.588345e+20,-2.932274e+26,-7.844247e+26,-2.690801e+22,-5.487366e+22,4.618415e+28,-1.426866e+28
9,7.022516e+19,0.000000e+00,-3.642978e+21,-3.008366e-03,9.619526e+19,0.000000e+00,-5.277320e+20,1.200439e+00,1.439867e+29,-7.205711e+29,...,1.215446e+29,-1.929887e+29,8.589891e+19,3.293306e+19,8.720856e+24,8.978930e+24,6.293295e+20,6.110087e+20,-9.502075e+26,4.627464e+26


In [30]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = matrPhi_l[0].dot(matrQ_il[0][l]).dot(la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
print(la.norm(matrPhi_l[25], np.inf))

LinAlgError: singular matrix

In [30]:
matr_a = - copy.deepcopy(matrQ_il[0][0])
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1
matr_a = np.transpose(matr_a)
p0 = la.solve(matr_a, matr_b).reshape(-1)

print(p0)
print(p0.sum())

IndexError: list index out of range

In [31]:
vect_p_l = [p0]
for l in range(1, p_num):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(vect_p_l[l])

NameError: name 'p0' is not defined

In [32]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-28.110725,0.000000,20.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
1,20.000000,-28.110725,0.00000,20.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
2,0.000000,0.000000,-20.26325,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
3,0.000000,0.000000,20.00000,-20.26325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
4,5.582250,0.000000,0.00000,0.00000,-15.692975,0.000000,0.000000,0.000000,2.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,440.998200,0.000000,0.00000,0.00000,0.000000,-15.692975,0.000000,0.000000,0.00000,2.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.582250,0.000000,0.00000,0.00000,2.000000,0.000000,-15.692975,0.000000,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,440.998200,0.000000,0.00000,0.00000,0.000000,2.000000,0.000000,-15.692975,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,5.58225,0.00000,0.000000,0.000000,0.000000,0.000000,-443.26145,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,440.99820,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,-443.26145,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
matr_a = np.transpose(matrGamma)
matr_b = np.zeros((matr_a.shape[0], 1))
x = la.lstsq(matr_a, matr_b)[3]
vect_e = np.array([[1.] for i in range(0, x.shape[0])])
res = x.dot(vect_e)[0]
x /= res
print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

x =  [ 0.16664882  0.13778178  0.09753639  0.09537417  0.09494841  0.04665250
  0.04465977  0.04342233  0.04276691  0.03395629  0.03158904  0.03007703
  0.02191321  0.02187875  0.02159697  0.02112990  0.01512927  0.01121088
  0.00796851  0.00373772  0.00334295  0.00324881  0.00142412  0.00060284
  0.00058718  0.00052926  0.00025505  0.00003113]
x1 =  [ 0.16664882  0.13778178  0.09753639  0.09537417]
x2 =  [ 0.09494841  0.04665250  0.04465977  0.04342233  0.04276691  0.03395629
  0.03158904  0.03007703]
x3 =  [ 0.02191321  0.02187875  0.02159697  0.02112990  0.01512927  0.01121088
  0.00796851  0.00373772]


In [ ]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

In [ ]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

In [ ]:
print(la.norm(np.eye(matrQ_il[0][0].shape[0]).dot(matrQ_il[0][1]).dot(la.inv(-matrQ_il[1][1])), np.inf))

In [ ]:
result = np.sum(matrQv_0[0])
i = 0
for matrQk in matrQ_k:
    if i > 0:
        result += np.sum(matrQk[0])
    i += 1
print(result)